# 需要先转换下模型

```bash
python .\convert.py G:\code\pretrain_model_dir\llama-7b

$env:CMAKE_ARGS = "-DLLAMA_CUBLAS=on"
pip install llama-cpp-python --no-cache-dir --verbose
```

TODO: 感觉好像没跑在 GPU 上, 需要看下文档

In [2]:
from llama_cpp import Llama

In [6]:
help(Llama)

Help on class Llama in module llama_cpp.llama:

class Llama(builtins.object)
 |  Llama(model_path: str, *, n_gpu_layers: int = 0, main_gpu: int = 0, tensor_split: Union[List[float], NoneType] = None, vocab_only: bool = False, use_mmap: bool = True, use_mlock: bool = False, seed: int = 4294967295, n_ctx: int = 512, n_batch: int = 512, n_threads: Union[int, NoneType] = None, n_threads_batch: Union[int, NoneType] = None, rope_freq_base: float = 0.0, rope_freq_scale: float = 0.0, mul_mat_q: bool = True, f16_kv: bool = True, logits_all: bool = False, embedding: bool = False, last_n_tokens_size: int = 64, lora_base: Union[str, NoneType] = None, lora_scale: float = 1.0, lora_path: Union[str, NoneType] = None, numa: bool = False, chat_format: str = 'llama-2', verbose: bool = True, **kwargs)
 |  
 |  High-level Python wrapper for a llama.cpp model.
 |  
 |  Methods defined here:
 |  
 |  __call__(self, prompt: str, suffix: Union[str, NoneType] = None, max_tokens: int = 128, temperature: float =

In [3]:
model_path = r"G:\code\pretrain_model_dir\llama-7b\ggml-model-f16.gguf"
llm = Llama(model_path=model_path, n_gpu_layers=-1)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [5]:
output = llm("I look forward to", max_tokens=40, stop=["\n"], echo=True)
output

Llama.generate: prefix-match hit


{'id': 'cmpl-6e3205c9-9140-4671-b5a5-c62ce0877328',
 'object': 'text_completion',
 'created': 1697122412,
 'model': 'G:\\code\\pretrain_model_dir\\llama-7b\\ggml-model-f16.gguf',
 'choices': [{'text': 'I look forward to meeting you at our wedding!',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 5, 'completion_tokens': 8, 'total_tokens': 13}}

In [7]:
import time
import torch
from transformers import LlamaTokenizer, LlamaTokenizerFast

model_path = r"G:\code\pretrain_model_dir\llama-7b-hf"
tokenizer = LlamaTokenizer.from_pretrained(model_path)

# 记录每次生成的时间和 token 数量
time_list = []
token_list = []

query_list = [
    "I look forward to",
    "I love beijing , because",
]
with open("./data/query.txt", "r", encoding="utf-8") as f:
    query_list = f.readlines()

for query in query_list:
    start = time.time()
    output = llm(query, max_tokens=40, stop=["\n"], echo=True)
    end = time.time()
    print(f"query: {query}, time: {end - start}")
    print(output["choices"][0]["text"])

    time_list.append(end - start)
    inputs = tokenizer(query, return_tensors="pt")
    outputs = tokenizer(output["choices"][0]["text"], return_tensors="pt")
    token_list.append(outputs.input_ids.shape[1] - inputs.input_ids.shape[1])

print(time_list)
print(token_list)
# 计算每秒生成的 token 数量
print("每秒 token 数:", sum(token_list) / sum(time_list))

Llama.generate: prefix-match hit


query: He opened his eyes and gaspe
, time: 1.1316921710968018
He opened his eyes and gaspe
He opened his eyes and gasped. What the hell was that? He saw a white light flashing in front of him, he looked up and saw some kind of flying object. He heard


Llama.generate: prefix-match hit


query: She ran as fast as she coul
, time: 0.9549820423126221
She ran as fast as she coul
She ran as fast as she could, and when she got to the end of the street she stopped. She looked around for a little while but didn’t see anyone. In fact, no one


Llama.generate: prefix-match hit


...

query: He had a mission. A secret one
, time: 0.4280893802642822
He had a mission. A secret one
Grabbing the arm of his friend, he shook him awake.


Llama.generate: prefix-match hit


query: She was a princess. But not by choice
, time: 0.962332010269165
She was a princess. But not by choice
"I'll tell you what happens to the daughter of Queen Victoria," says Diana Mishkova, "She becomes a prisoner in her own home." For Diana and her sister Maria


Llama.generate: prefix-match hit


query: He was a thief. A master one
, time: 0.954064130783081
He was a thief. A master one
The man who was in the lead had his back turned to him but he didn't feel he should risk waiting any longer for things to get worse - and so he leaped forward at the man
[1.1316921710968018, 0.9549820423126221, 0.7493360042572021, 0.9612278938293457, 0.9492988586425781, 0.9490706920623779, 0.9651272296905518, 0.9527075290679932, 0.885735034942627, 0.9589717388153076, 0.9472537040710449, 0.954035758972168, 0.9653947353363037, 0.7694878578186035, 0.9547691345214844, 0.5894136428833008, 0.9366207122802734, 0.9536046981811523, 0.9619050025939941, 0.9590346813201904, 0.9631884098052979, 0.3790779113769531, 0.9321498870849609, 0.9551591873168945, 0.9551372528076172, 0.959038257598877, 0.9589338302612305, 0.4280893802642822, 0.962332010269165, 0.954064130783081]
[40, 40, 30, 40, 40, 40, 40, 40, 36, 40, 40, 40, 40, 31, 40, 23, 38, 40, 40, 39, 40, 14, 38, 40, 40, 40, 40, 16, 40, 40]
每秒 token 数: 41.0828939